In [7]:
from peft import get_peft_model, LoraConfig, TaskType
from src.models import MODELS_MAPPING

import torch

In [4]:
model_class = MODELS_MAPPING.get(
    "llama", {}
    ).get("mntp")

In [5]:
model_class

src.models.llama.biLlamaForMaskedNTP

In [8]:
model = model_class.from_pretrained(
    "meta-llama/Llama-3.2-3B",
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
model.lm_head.state_dict()

In [9]:
lora_config = LoraConfig(
    r=64,  # the dimension of the low-rank matrices
    lora_alpha=128, # scaling factor for LoRA activations vs pre-trained weight activations
    lora_dropout=0.05, 
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM,
    target_modules=['o_proj', 'v_proj', "q_proj", "k_proj", "gate_proj", "down_proj", "up_proj"]
) 

model = get_peft_model(model, lora_config)
# Trainable Parameters
model.print_trainable_parameters()

trainable params: 97,255,424 || all params: 3,310,005,248 || trainable%: 2.9382


In [15]:
model.lm_head.state_dict()

OrderedDict([('weight',
              tensor([[ 0.0110,  0.0111,  0.0132,  ..., -0.0026, -0.0189,  0.0067],
                      [ 0.0134,  0.0013,  0.0210,  ...,  0.0007,  0.0327, -0.0026],
                      [ 0.0251,  0.0204,  0.0284,  ..., -0.0012, -0.0008, -0.0079],
                      ...,
                      [-0.0083,  0.0036,  0.0050,  ..., -0.0024, -0.0020, -0.0052],
                      [-0.0083,  0.0036,  0.0050,  ..., -0.0024, -0.0020, -0.0052],
                      [-0.0083,  0.0036,  0.0050,  ..., -0.0024, -0.0020, -0.0052]],
                     dtype=torch.float16))])

In [17]:
model.lm_head.weight.requires_grad

False

In [23]:
from transformers import (
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig
)

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-3B"
)

In [26]:
tokenizer.mask_token

False

In [27]:
tokenizer.add

AttributeError: PreTrainedTokenizerFast has no attribute add